In [6]:
using Random, Statistics

"""
mean_matched_ff(counts; nbins=20, n_reps=50, seed=0)

counts: Array{<:Real,3} with dims (T, n_units, n_trials)
Returns: Vector{Float64} of length T (mean-matched FF per time)
"""
function mean_matched_ff(counts; nbins=20, n_reps=50, seed=0)
    T, U, R = size(counts)

    # per-time per-unit mean/var across trials
    means = [mean(@view counts[t,u,:]) for t in 1:T, u in 1:U]
    vars  = [var(@view counts[t,u,:])  for t in 1:T, u in 1:U]

    # bin edges from pooled positive means
    pool = [μ for μ in vec(means) if μ > 0]
    mn, mx = minimum(pool), maximum(pool)
    edges = collect(range(mn, mx, length=nbins+1))
    nb = length(edges) - 1

    # per-time, per-bin members (unit indices)
    bin_members = [[Int[] for _ in 1:nb] for _ in 1:T]
    for t in 1:T, u in 1:U
        μ = means[t,u]
        if μ > 0
            b = clamp(searchsortedlast(edges, μ), 1, nb)
            push!(bin_members[t][b], u)
        end
    end
    @info bin_members

    # greatest common per-bin counts across times (fixed)
    common = [minimum([length(bin_members[t][b]) for t in 1:T]) for b in 1:nb]

    # repetitions
    ff = zeros(Float64, T)
    rng = MersenneTwister(seed)
    for rep in 1:n_reps
        for t in 1:T
            sel = Int[]
            for b in 1:nb
                h = common[b]
                if h > 0
                    members = bin_members[t][b]
                    idx = (length(members) <= h) ? collect(1:length(members)) : randperm(rng, length(members))[1:h]
                    append!(sel, members[idx])
                end
            end
            if !isempty(sel)
                μs = means[t, sel]
                vs = vars[t, sel]
                ff[t] += mean(vs ./ μs)
            end
        end
    end
    ff ./= n_reps
    return ff
end


mean_matched_ff

In [16]:
using Random, Distributions
rng = MersenneTwister(42)
T, U, R = 5, 100, 1
counts = rand(rng, Poisson(5.0), T, U, R)

ff = mean_matched_ff(counts; nbins=15, n_reps=50, seed=123)
println(ff)


[NaN, NaN, NaN, NaN, NaN]


┌ Info: [[[3, 10, 49, 60, 79, 86], [2, 9, 13, 26, 33, 47, 57, 59, 66, 85], [19, 23, 25, 39, 52, 56, 68, 69, 73, 80, 83, 87, 90, 95], [16, 17, 21, 30, 37, 43, 51, 53, 54, 64, 67, 75, 76, 91, 94], [6, 8, 20, 36, 38, 40, 44, 45, 50, 61, 65, 70, 74, 77, 78, 84, 88, 93], [5, 7, 12, 15, 18, 22, 24, 27, 31, 41, 58, 62, 96, 99], [4, 11, 29, 32, 42, 55, 71, 72, 97], Int64[], [34, 48, 92, 100], [14, 35, 82, 89], [28, 46, 98], [1, 81], Int64[], Int64[], Int64[]], [[7, 93], [9, 13, 21, 42, 68, 75, 88], [19, 34, 43, 48, 60, 64, 66, 69, 76, 80, 84, 86, 92, 98], [2, 5, 10, 11, 12, 23, 27, 28, 31, 32, 33, 36, 38, 40, 41, 47, 49, 51, 61, 85, 94, 95, 96], [6, 8, 14, 17, 30, 45, 50, 55, 58, 65, 67, 71, 73, 77, 82, 83, 87, 91, 99], [1, 15, 18, 22, 26, 29, 39, 46, 56, 57, 59, 62, 79, 100], [4, 16, 20, 35, 52, 54], Int64[], [3, 24, 53, 63, 81, 90], [25, 70, 74], [37, 44, 72, 89], [78], Int64[], Int64[], Int64[]], [[78, 100], [6, 14, 35, 40, 69, 70, 81, 87], [5, 8, 10, 20, 21, 22, 28, 38, 43, 46, 52, 58, 59,

In [54]:
using Random, Statistics

"""
mean_matched_ff(counts; nbins=20, n_reps=50, seed=0)

- counts: (T,P,R) where P are (unit×condition) pairs and R are repeats,
          or (T,U,C,R) which will be reshaped to (T,P,R) with P=U*C.
- returns: Vector{Float64} of length T (mean-matched FF per time).
"""
function mean_matched_ff(counts; nbins=20, n_reps=50, seed=0)
    A = counts
    nd = ndims(A)
    if nd == 4
        T, U, C, R = size(A)
        A = reshape(A, T, U*C, R)  # (T,P,R)
    elseif nd == 3
        T, P, R = size(A)
    else
        error("counts must be (T,P,R) or (T,U,C,R)")
    end
    T, P, R = size(A)

    # per-time, per-pair stats across repeats
    means = [mean(@view A[t,p,:]) for t in 1:T, p in 1:P]
    vars  = [var(@view A[t,p,:])  for t in 1:T, p in 1:P]

    # pooled positive means → common bins
    pool = [μ for μ in vec(means) if μ > 0]
    mn, mx = minimum(pool), maximum(pool)
    edges = collect(range(mn, mx, length=nbins+1))
    nb = length(edges) - 1

    # collect members per time/bin (indices into pairs)
    bins = [[Int[] for _ in 1:nb] for _ in 1:T]
    for t in 1:T, p in 1:P
        μ = means[t,p]
        if μ > 0
            b = clamp(searchsortedlast(edges, μ), 1, nb)
            push!(bins[t][b], p)
        end
    end

    # greatest common per-bin counts across times
    common = [minimum([length(bins[t][b]) for t in 1:T]) for b in 1:nb]

    # subsample repeatedly and average FF
    rng = MersenneTwister(seed)
    ff = zeros(Float64, T)
    for rep in 1:n_reps
        for t in 1:T
            sel = Int[]
            for b in 1:nb
                h = common[b]
                if h > 0
                    members = bins[t][b]
                    idx = (length(members) <= h) ? collect(1:length(members)) : randperm(rng, length(members))[1:h]
                    append!(sel, members[idx])
                end
            end
            if !isempty(sel)
                μs = means[t, sel]
                vs = vars[t,  sel]
                ff[t] += mean(vs ./ μs)
            end
        end
    end
    ff ./= n_reps
    return ff
end


mean_matched_ff

In [60]:
using Random, Distributions
rng = MersenneTwister(42)
T, P, R = 5, 10, 1000
counts = rand(rng, Poisson(5.0), T, P, R)

ff = mean_matched_ff(counts; nbins=15, n_reps=50, seed=123)
println(ff)


[0.9697876078146543, 0.9774042091553049, 1.0621200042357726, 0.9787029971119423, 0.9743418615797524]


In [61]:
include("../src/Analysis.jl")

Main.Analysis